In [3]:
import sys
import pandas as pd
from datetime import date
import os

In [4]:
def update_prices(base_list_prices_path:str, new_list_prices_path:str):
    df_base = pd.read_excel(base_list_prices_path)
    new_df = pd.read_excel(new_list_prices_path)
    new_df['30%'] = new_df['PVP']*0.3
    datastr=update_date_df(new_df, '01-02-2023')
    updated_prices_list = pd.concat([df_base, new_df], ignore_index=True, axis=0).drop_duplicates(keep='last', subset=['COD. ORIGEN'])
    updated_prices_list.to_excel('result.xlsx', index=False, sheet_name=datastr)

### Este es el archivo base de la lista de precios

In [5]:
base = pd.read_excel("LP30.11.2022.xlsx", index_col=None)

FileNotFoundError: [Errno 2] No such file or directory: 'LP30.11.2022.xlsx'

### Validación si la lista base tiene el formato requerido


In [247]:
columnas = base.columns.values
valores = ["COD. ORIGEN", "LINEA","DESCRIPCION", "MASTER PACK", "PVP", "30%", "Fecha de actualizacion"]
for i in range(6):
    if valores[i] != columnas[i]:
        message = f"El campo {columnas[i]} no coincide con {valores[i]}"
        raise Exception(message)
print("Formato correcto")

Formato correcto


### Validación para detectar registros que no tiene COD. ORIGEN

In [248]:
if len(base[base["COD. ORIGEN"].isnull()].index):
    raise Exception("Las siguientes filas no tienen codigo de origen")
print("Todas las filas tienen codigo de origen")

Todas las filas tienen codigo de origen


### Validación para detectar registros del mismo SKU repetidos
En la lista de precios base no deben existir más de un registro por SKU, en otras palabras, no deberían existir dos registros con el mismo codigo de origen **(COD. ORIGEN)**.

In [2]:
df = base.groupby(["COD. ORIGEN"], as_index=False).size()
df[df['size'] != 1].index


NameError: name 'base' is not defined

In [249]:
df = base.groupby(["COD. ORIGEN"], as_index=False).size()
if len(df[df['size'] != 1].index) > 0:
    raise Exception("Existen SKU repetidos")
print("Validación de SKU no repetidos")

Validación de SKU no repetidos


### Cargamos la nueva lista de precios


In [269]:
nueva_lista = pd.read_excel('C:/Users/abernabel/Desktop/virus/nuevo.xlsx')
nueva_lista

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP
0,70902,AIR STUNT SHOW: AVIONETA DE TIGRE,4,179.0
1,70833,AIR STUNT SHOW HELICÓPTERO CON EQUIPO DE FILMA...,4,219.0
2,70550,ESPECTÁCULO DE ACROBACIAS CAMIÓN MONSTRUO TIBURÓN,2,399.0
3,70831,ESPECTÁCULO DE ACROBACIAS AÉREAS BIPLANO PHOENIX,4,499.0
4,70832,ESPECTÁCULO DE ACROBACIAS AÉREAS EAGLE JET,4,499.0
...,...,...,...,...
188,70817,STARTER PACK POLICÍA: ENTRENAMIENTO,4,139.0
189,70820,STARTER PACK STUNTSHOW QUAD CON RAMPA DE FUEGO,4,139.0
190,70009,PLAYMOBIL SUPERSET GALAXIA,4,139.0
191,70645,LLAVERO A-TEAM B.A.,10,39.9


### Le damos la estructura correspondiente
COD. ORIGEN DESCRIPCION MASTAR PACK PVP 30% FECHA

In [270]:
nueva_lista = pd.merge(nueva_lista, base[["COD. ORIGEN", "LINEA"]], on=["COD. ORIGEN"],how="inner")
nueva_lista

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP,LINEA
0,71090,PLAYMOBIL CITY ACTION: CARRO DE BOMBEROS VELOC...,5,64.9,TOTAL PLAYMOBIL
1,71093,PLAYMOBIL CITY ACTION: CARRO DE CARRERAS VELOC...,5,64.9,TOTAL PLAYMOBIL
2,71092,PLAYMOBIL CITY ACTION: CARRO DE POLICÍA VELOCI...,5,64.9,TOTAL PLAYMOBIL
3,71091,PLAYMOBIL CITY ACTION: CARRO DE RESCATE VELOCI...,5,64.9,TOTAL PLAYMOBIL
4,70674,PLAYMOBIL CITY LIFE SET DE REGALO TALLER DE BI...,8,69.9,TOTAL PLAYMOBIL
...,...,...,...,...,...
74,70597,PLAYMOBIL PROMO SOLDADOR,5,29.9,TOTAL PLAYMOBIL
75,70378,PLAYMOBIL CABALLERO ENANO,5,29.9,TOTAL PLAYMOBIL
76,70035,PLAYMOBIL STARTERPACK AVENTURA EN CANOA,8,89.9,TOTAL PLAYMOBIL
77,70033,PLAYMOBIL STARTERPACK CARRUAJES CABALLITOS DE MAR,8,89.9,TOTAL PLAYMOBIL


In [277]:
nueva_lista = nueva_lista.reindex(columns=["COD. ORIGEN", "LINEA","DESCRIPCION", "MASTER PACK", "PVP"])

In [278]:
nueva_lista["30%"] = nueva_lista["PVP"]*0.7
nueva_lista["FECHA DE ACTUALIZACION"] = "19/01/2023"
#nueva_lista = nueva_lista.drop("Fecha de actualizacion", axis=1)
nueva_lista

,COD. ORIGEN,LINEA,DESCRIPCION,MASTER PACK,PVP,30%,FECHA DE ACTUALIZACION
0,71090,TOTAL PLAYMOBIL,PLAYMOBIL CITY ACTION: CARRO DE BOMBEROS VELOC...,5,64.9,45.43,19/01/2023
1,71093,TOTAL PLAYMOBIL,PLAYMOBIL CITY ACTION: CARRO DE CARRERAS VELOC...,5,64.9,45.43,19/01/2023
2,71092,TOTAL PLAYMOBIL,PLAYMOBIL CITY ACTION: CARRO DE POLICÍA VELOCI...,5,64.9,45.43,19/01/2023
3,71091,TOTAL PLAYMOBIL,PLAYMOBIL CITY ACTION: CARRO DE RESCATE VELOCI...,5,64.9,45.43,19/01/2023
4,70674,TOTAL PLAYMOBIL,PLAYMOBIL CITY LIFE SET DE REGALO TALLER DE BI...,8,69.9,48.93,19/01/2023
...,...,...,...,...,...,...,...
74,70597,TOTAL PLAYMOBIL,PLAYMOBIL PROMO SOLDADOR,5,29.9,20.93,19/01/2023
75,70378,TOTAL PLAYMOBIL,PLAYMOBIL CABALLERO ENANO,5,29.9,20.93,19/01/2023
76,70035,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK AVENTURA EN CANOA,8,89.9,62.93,19/01/2023
77,70033,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK CARRUAJES CABALLITOS DE MAR,8,89.9,62.93,19/01/2023


### Creamos una lista temporal de concatenacion
En esta nueva lista temporal vamos a unir las dos listas en una sola; sin embargo, esto ocasionará duplicidad de registros

In [279]:
nueva_lista_tmp = pd.concat([base, nueva_lista])
nueva_lista_tmp

,COD. ORIGEN,LINEA,DESCRIPCION,MASTER PACK,PVP,30%,FECHA DE ACTUALIZACION
0,1416228,TOTAL TEAMSTERZ,"Teamsterz autos de metal 3"" en blister",48,6.9,4.83,2022-11-30 00:00:00
1,1416576,TOTAL TEAMSTERZ,Teamsterz lanzador ataque dinosaurio incl 3 a...,12,39.9,27.93,2022-11-30 00:00:00
2,1416803,TOTAL TEAMSTERZ,Teamsterz camión para guardar 12 autos incl 4 ...,12,59.9,41.93,2022-11-30 00:00:00
3,1416211,TOTAL TEAMSTERZ,"Teamsterz autos de metal 3"" pack de 2",18,13.8,9.66,2022-11-30 00:00:00
4,1416849,TOTAL TEAMSTERZ,Teamsterz lanzador con caldero slime incl 1 au...,12,39.9,27.93,2022-11-30 00:00:00
...,...,...,...,...,...,...,...
74,70597,TOTAL PLAYMOBIL,PLAYMOBIL PROMO SOLDADOR,5,29.9,20.93,19/01/2023
75,70378,TOTAL PLAYMOBIL,PLAYMOBIL CABALLERO ENANO,5,29.9,20.93,19/01/2023
76,70035,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK AVENTURA EN CANOA,8,89.9,62.93,19/01/2023
77,70033,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK CARRUAJES CABALLITOS DE MAR,8,89.9,62.93,19/01/2023


### Examinando la lista temporal
El siguiente codigo myestra todos los registros repetidos

In [281]:
t = nueva_lista_tmp.groupby(['COD. ORIGEN'], as_index=False).size()
t[t['size'] > 1]

,COD. ORIGEN,size
22,5649,2
83,9461,2
87,10127,2
120,70009,2
124,70033,2
...,...,...
257,70930,2
259,71090,2
260,71091,2
261,71092,2


### Eliminamos registros más antiguos (precios antiguos)

In [282]:
nueva_lista_tmp.drop_duplicates(keep='last', subset=['COD. ORIGEN'])


,COD. ORIGEN,LINEA,DESCRIPCION,MASTER PACK,PVP,30%,FECHA DE ACTUALIZACION
0,1416228,TOTAL TEAMSTERZ,"Teamsterz autos de metal 3"" en blister",48,6.9,4.83,2022-11-30 00:00:00
1,1416576,TOTAL TEAMSTERZ,Teamsterz lanzador ataque dinosaurio incl 3 a...,12,39.9,27.93,2022-11-30 00:00:00
2,1416803,TOTAL TEAMSTERZ,Teamsterz camión para guardar 12 autos incl 4 ...,12,59.9,41.93,2022-11-30 00:00:00
3,1416211,TOTAL TEAMSTERZ,"Teamsterz autos de metal 3"" pack de 2",18,13.8,9.66,2022-11-30 00:00:00
4,1416849,TOTAL TEAMSTERZ,Teamsterz lanzador con caldero slime incl 1 au...,12,39.9,27.93,2022-11-30 00:00:00
...,...,...,...,...,...,...,...
74,70597,TOTAL PLAYMOBIL,PLAYMOBIL PROMO SOLDADOR,5,29.9,20.93,19/01/2023
75,70378,TOTAL PLAYMOBIL,PLAYMOBIL CABALLERO ENANO,5,29.9,20.93,19/01/2023
76,70035,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK AVENTURA EN CANOA,8,89.9,62.93,19/01/2023
77,70033,TOTAL PLAYMOBIL,PLAYMOBIL STARTERPACK CARRUAJES CABALLITOS DE MAR,8,89.9,62.93,19/01/2023


### Ordenar la lista de precios
Finalmente ordenamos la lista de precios en base a la línea

In [283]:
    nueva_lista_tmp = nueva_lista_tmp.sort_values("LINEA", ascending=False).reset_index()
nueva_lista_tmp

,index,COD. ORIGEN,LINEA,DESCRIPCION,MASTER PACK,PVP,30%,FECHA DE ACTUALIZACION
0,434,YW211135,TOTAL WACE RACERS,Wave Racers pista Mega Match 3 vueltas en espiral,6,169.0,118.30,2022-11-30 00:00:00
1,443,EU211136C,TOTAL WACE RACERS,WAVE RACERS PISTA ESPIRAL FRENCY CON 9 VUELTAS...,6,179.0,125.30,2022-11-30 00:00:00
2,435,EU211131C,TOTAL WACE RACERS,WAVE RACERS PISTA CON SUPER LOOP AÉREO,12,79.9,55.93,2022-11-30 00:00:00
3,436,EU211132C,TOTAL WACE RACERS,WAVE RACERS CIRCUITO DE SUPER VELOCIDAD,12,69.9,48.93,2022-11-30 00:00:00
4,438,YW211121-4,TOTAL WACE RACERS,WAVE RACERS AUTO CON SENSOR ACELERADOR RIVAL 300X,NaN,39.9,27.93,2022-11-30 00:00:00
...,...,...,...,...,...,...,...,...
569,459,16-89015,TOTAL BURAGO JUNIOR,CHAPOTEA Y JUEGA BARCO DE BOMBEROS,6,79.9,55.93,2022-11-30 00:00:00
570,460,16-89009,TOTAL BURAGO JUNIOR,CHAPOTEA Y JUEGA SET DE REGALO DE CAPITAN JR,4,149.0,104.30,2022-11-30 00:00:00
571,153,BBHL1B,TOTAL BARBIE CABELLO,BARBIE APLICADOR DE CUENTAS,6,59.9,41.93,2022-11-30 00:00:00
572,154,BBHL2B,TOTAL BARBIE CABELLO,BARBIE TRENZADOR DE CABELLO,6,59.9,41.93,2022-11-30 00:00:00


### Importamos la lista de precios

In [285]:
nueva_lista_tmp.to_excel('lista_precios-01012023.xlsx', index=False, sheet_name="01.01.2023")